# Exploratory Data Analysis (EDA) for Vehicle Specifications

In [ ]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(os.path.join(os.path.dirname(__file__), '../../.env'))

# Database connection details
db_host = os.getenv('POSTGRES_HOST')
db_port = os.getenv('POSTGRES_PORT')
db_name = os.getenv('POSTGRES_DB')
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')

conn_string = f"host={db_host} port={db_port} dbname={db_name} user={db_user} password={db_password}"

try:
    conn = psycopg2.connect(conn_string)
    print("Successfully connected to the database!")
except psycopg2.Error as e:
    print(f"Error connecting to the database: {e}")
    conn = None

## 1. Querying `vehicle_trims`

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exploratory Data Analysis (EDA) for Vehicle Specifications"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import pandas as pd\n",
    "import psycopg2\n",
    "from dotenv import load_dotenv\n",
    "\n",
    "# Load environment variables from .env file\n",
    "load_dotenv(os.path.join(os.path.dirname(__file__), '../../.env'))\n",
    "\n",
    "# Database connection details\n",
    "db_host = os.getenv('POSTGRES_HOST')\n",
    "db_port = os.getenv('POSTGRES_PORT')\n",
    "db_name = os.getenv('POSTGRES_DB')\n",
    "db_user = os.getenv('POSTGRES_USER')\n",
    "db_password = os.getenv('POSTGRES_PASSWORD')\n",
    "\n",
    "conn_string = f\"host={db_host} port={db_port} dbname={db_name} user={db_user} password={db_password}\"\n",
    "\n",
    "try:\n",
    "    conn = psycopg2.connect(conn_string)\n",
    "    print(\"Successfully connected to the database!\")\n",
    "except psycopg2.Error as e:\n",
    "    print(f\"Error connecting to the database: {e}\")\n",
    "    conn = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Querying `vehicle_trims`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if conn:\n",
    "    query = \"SELECT * FROM vehicle_trims LIMIT 5;\"\n",
    "    df_trims = pd.read_sql(query, conn)\n",
    "    print(\"First 5 rows of vehicle_trims:\")\n",
    "    display(df_trims)\n",
    "else:\n",
    "    print(\"Database connection not established. Cannot query data.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Querying `performance_specs`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if conn:\n",
    "    query = \"SELECT * FROM performance_specs LIMIT 5;\"\n",
    "    df_performance = pd.read_sql(query, conn)\n",
    "    print(\"First 5 rows of performance_specs:\")\n",
    "    display(df_performance)\n",
    "else:\n",
    "    print(\"Database connection not established. Cannot query data.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Joining Data (Example: Vehicle Trims with Performance Specs)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if conn:\n",
    "    query = \"\"\"\n",
    "    SELECT\n",
    "        vt.make,\n",
    "        vt.model,\n",
    "        vt.year,\n",
    "        vt.trim_name,\n",
    "        ps.horsepower,\n",
    "        ps.fuel_economy_combined_l_per_100km\n",
    "    FROM\n",
    "        vehicle_trims vt\n",
    "    JOIN\n",
    "        performance_specs ps ON vt.vehicle_trim_id = ps.performance_id\n",
    "    LIMIT 5;\n",
    "    \"\"\"\n",
    "    df_joined = pd.read_sql(query, conn)\n",
    "    print(\"First 5 rows of joined data (Trims & Performance):\")\n",
    "    display(df_joined)\n",
    "    conn.close()\n",
    "else:\n",
    "    print(\"Database connection not established. Cannot query data.\")"
   ]
 },
 {
  "cell_type": "markdown",
  "metadata": {},
  "source": [
   "## 4. Querying JSONB Fields (additional_features_json)"
  ]
 },
 {
  "cell_type": "code",
  "execution_count": null,
  "metadata": {},
  "outputs": [],
  "source": [
   "if conn:\n",
   "    # Re-establish connection if closed\n",
   "    if conn.closed:\n",
   "        conn = psycopg2.connect(conn_string)\n",
   "\n",
   "    # Example 1: Basic field access\n",
   "    query = \"\"\"\n",
   "    SELECT \n",
   "        trim_name,\n",
   "        additional_features_json->>'sunroof' as has_sunroof,\n",
   "        additional_features_json->>'parking_assist' as has_parking_assist\n",
   "    FROM vehicle_trims\n",
   "    LIMIT 5;\n",
   "    \"\"\"\n",
   "    df_json1 = pd.read_sql(query, conn)\n",
   "    print(\"\\nExample 1: Basic JSONB field access:\")\n",
   "    display(df_json1)\n",
   "\n",
   "    # Example 2: Filtering on JSONB attributes\n",
   "    query = \"\"\"\n",
   "    SELECT \n",
   "        make,\n",
   "        model,\n",
   "        trim_name,\n",
   "        additional_features_json->>'sunroof' as sunroof_type\n",
   "    FROM vehicle_trims\n",
   "    WHERE additional_features_json->>'sunroof' IS NOT NULL\n",
   "    LIMIT 5;\n",
   "    \"\"\"\n",
   "    df_json2 = pd.read_sql(query, conn)\n",
   "    print(\"\\nExample 2: Filtering vehicles with sunroof:\")\n",
   "    display(df_json2)\n",
   "\n",
   "    # Example 3: Aggregation with JSONB data\n",
   "    query = \"\"\"\n",
   "    SELECT \n",
   "        make,\n",
   "        COUNT(*) as total_models,\n",
   "        COUNT(*) FILTER (WHERE additional_features_json->>'sunroof' IS NOT NULL) as with_sunroof,\n",
   "        ROUND(100.0 * COUNT(*) FILTER (WHERE additional_features_json->>'sunroof' IS NOT NULL) / COUNT(*), 1) as pct_with_sunroof\n",
   "    FROM vehicle_trims\n",
   "    GROUP BY make\n",
   "    ORDER BY total_models DESC\n",
   "    LIMIT 5;\n",
   "    \"\"\"\n",
   "    df_json3 = pd.read_sql(query, conn)\n",
   "    print(\"\\nExample 3: Aggregation by make with JSONB features:\")\n",
   "    display(df_json3)\n",
   "\n",
   "    # Example 4: Dynamic feature listing\n",
   "    query = \"\"\"\n",
   "    SELECT \n",
   "        jsonb_object_keys(additional_features_json) as feature_name,\n",
   "        COUNT(*) as frequency\n",
   "    FROM vehicle_trims\n",
   "    GROUP BY feature_name\n",
   "    ORDER BY frequency DESC;\n",
   "    \"\"\"\n",
   "    df_json4 = pd.read_sql(query, conn)\n",
   "    print(\"\\nExample 4: Frequency of all JSONB features:\")\n",
   "    display(df_json4)\n",
   "\n",
   "    conn.close()\n",
   "else:\n",
   "    print(\"Database connection not established. Cannot query data.\")"
  ]
 }
],
"metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

## 2. Querying `performance_specs`

In [ ]:
if conn:
    query = "SELECT * FROM performance_specs LIMIT 5;"
    df_performance = pd.read_sql(query, conn)
    print("First 5 rows of performance_specs:")
    display(df_performance)
else:
    print("Database connection not established. Cannot query data.")

## 3. Joining Data (Example: Vehicle Trims with Performance Specs)

In [ ]:
if conn:
    query = """
    SELECT
        vt.make,
        vt.model,
        vt.year,
        vt.trim_name,
        ps.horsepower,
        ps.fuel_economy_combined_l_per_100km
    FROM
        vehicle_trims vt
    JOIN
        performance_specs ps ON vt.vehicle_trim_id = ps.performance_id
    LIMIT 5;
    """
    df_joined = pd.read_sql(query, conn)
    print("First 5 rows of joined data (Trims & Performance):")
    display(df_joined)
    conn.close()
else:
    print("Database connection not established. Cannot query data.")